Importing all required libraries:

In [ ]:
from google.colab import drive

import numpy as np
import pandas as pd

import cv2
from os import listdir

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Model, load_model
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.applications import ResNet101, ResNet152,  ResNet50V2, ResNet101V2, ResNet152V2, DenseNet121, DenseNet169, DenseNet201, NASNetLarge, ResNet50

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

Mounting Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Annotating the new images manually, not using the annotation tool as 10 images can be annotated easily like this. 
Importing the new images and adding them to main dataframe (df):

In [ ]:
# Annotating the new images manually
data = {'file_name': ['image1.jpg','image2.jpg','image3.jpg','image4.jpg','image5.jpg','image6.jpg','image7.jpg','image8.jpg','image9.jpg','image10.jpg'],
        'wrinkles': [0,1,1,0,0,0,0,0,1,0],
        'freakles': [1,1,0,0,0,0,0,1,1,0],
        'glasses': [0,0,0,0,1,0,0,0,2,0],
        'hair color': [0,8,1,0,1,5,3,1,1,6],
        'hair top': [2,3,1,1,2,2,2,3,2,1]
        }

df = pd.DataFrame(data, columns= ['file_name', 'wrinkles','freakles','glasses','hair color','hair top'])


# importing all the new images from Drive using open-cv2 and adding them to our dataframe
dir_struct = '/content/drive/MyDrive/new images/'
image_list = [] # this list will hold all the images
#for i in range(1):
for i in range(df.shape[0]): # read the image one by one
    image = cv2.imread(dir_struct+df.iloc[i]['file_name'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR to RGB convert
    image = cv2.resize(image, (96,96), interpolation = cv2.INTER_AREA) # resizing all images to 32x32: NOT ideal but need to be uniform in order to apply the algorithm
                                                                                                                        # this actually ensures all the features [pixel values] will be of same size
    image_list.append(image) # appending the read image to a list

df['images'] = image_list

df

,file_name,wrinkles,freakles,glasses,hair color,hair top,images
0,image1.jpg,0,1,0,0,2,"[[[96, 80, 70], [102, 85, 77], [58, 46, 44], [..."
1,image2.jpg,1,1,0,8,3,"[[[245, 248, 253], [245, 248, 253], [245, 248,..."
2,image3.jpg,1,0,0,1,1,"[[[49, 47, 44], [43, 39, 36], [37, 33, 29], [3..."
3,image4.jpg,0,0,0,0,1,"[[[118, 114, 122], [117, 113, 121], [118, 113,..."
4,image5.jpg,0,0,1,1,2,"[[[71, 62, 57], [72, 63, 58], [73, 64, 59], [7..."
5,image6.jpg,0,0,0,5,2,"[[[207, 213, 230], [208, 214, 231], [209, 215,..."
6,image7.jpg,0,0,0,3,2,"[[[59, 50, 52], [58, 50, 53], [59, 52, 55], [6..."
7,image8.jpg,0,1,0,1,3,"[[[79, 45, 42], [84, 46, 43], [89, 48, 44], [9..."
8,image9.jpg,1,1,2,1,2,"[[[245, 248, 253], [245, 248, 253], [245, 248,..."
9,image10.jpg,0,0,0,6,1,"[[[147, 95, 84], [149, 96, 84], [150, 96, 84],..."


Preprocessing:

In [ ]:
# manually converting it to the shape which will be acceptable to the model
test_images = df[['images']]
test_images['images'] = test_images['images'].apply(lambda x:x.reshape(1, 96,96,3)[0])

test_images = np.reshape([x for x in test_images['images'].values], (10, 96,96,3))

test_images = test_images.astype('float32')

# Normalizing
test_images = test_images / 255.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Importing trained model and its weights from Drive and then displaying the Evaluation Metrics
1. Wrinkles Model:

In [ ]:
# here you need to specify the saved model from before
filepath = "/content/drive/MyDrive/VGG16-wrinkles-10-0.6750.h5"

# load the model
model = load_model(filepath)


y_pred1_wrinkles = model.predict(test_images)
y_pred_wrinkles = np.argmax(y_pred1_wrinkles, axis=1) 

test_loss, test_acc = model.evaluate(test_images,  df[['freakles']], verbose=2)

print('These are the scores for the wrinkles attribute------------------------------------------------->')
print('Precision score: ')
print(precision_score(df[['wrinkles']], y_pred_wrinkles , average="micro"))
print('Recall score: ')
print(recall_score(df[['wrinkles']], y_pred_wrinkles , average="micro"))

print('F1 score (macro): ')
print(f1_score(df[['wrinkles']], y_pred_wrinkles , average="macro"))
print('F1 score (micro): ')
print(f1_score(df[['wrinkles']], y_pred_wrinkles , average="micro"))
print('F1 score (weighted): ')
print(f1_score(df[['wrinkles']], y_pred_wrinkles , average="weighted"))

print('Test accuracy: ')
print(test_acc)

1/1 - 1s - loss: 0.8409 - accuracy: 0.6000
These are the scores for the wrinkles attribute------------------------------------------------->
Precision score: 
0.7
Recall score: 
0.7
F1 score (macro): 
0.4117647058823529
F1 score (micro): 
0.7
F1 score (weighted): 
0.5764705882352941
Test accuracy: 
0.6000000238418579


2. Freakles Model:

In [ ]:
# here you need to specify the saved model from before
filepath = "/content/drive/MyDrive/VGG16-freakles-10-0.7300.h5"

# load the model
model = load_model(filepath)


y_pred1_freakles = model.predict(test_images)
y_pred_freakles = np.argmax(y_pred1_freakles, axis=1) 

test_loss, test_acc = model.evaluate(test_images,  df[['freakles']], verbose=2)

print('These are the scores for the freakles attribute------------------------------------------------->')
print('Precision score: ')
print(precision_score(df[['freakles']], y_pred_freakles , average="micro"))
print('Recall score: ')
print(recall_score(df[['freakles']], y_pred_freakles , average="micro"))

print('F1 score (macro): ')
print(f1_score(df[['freakles']], y_pred_freakles , average="macro"))
print('F1 score (micro): ')
print(f1_score(df[['freakles']], y_pred_freakles , average="micro"))
print('F1 score (weighted): ')
print(f1_score(df[['freakles']], y_pred_freakles , average="weighted"))

print('Test accuracy: ')
print(test_acc)


1/1 - 1s - loss: 0.7190 - accuracy: 0.6000
These are the scores for the freakles attribute------------------------------------------------->
Precision score: 
0.6
Recall score: 
0.6
F1 score (macro): 
0.37499999999999994
F1 score (micro): 
0.6
F1 score (weighted): 
0.4499999999999999
Test accuracy: 
0.6000000238418579


3. Glasses Model:

In [ ]:
# here you need to specify the saved model from before
filepath = "/content/drive/MyDrive/VGG16-glasses-10-0.8800.h5"

# load the model
model = load_model(filepath)


y_pred1_glasses = model.predict(test_images)
y_pred_glasses = np.argmax(y_pred1_glasses, axis=1) 

test_loss, test_acc = model.evaluate(test_images,  df[['glasses']], verbose=2)

print('These are the scores for the glasses attribute------------------------------------------------->')
print('Precision score: ')
print(precision_score(df[['glasses']], y_pred_glasses , average="micro"))
print('Recall score: ')
print(recall_score(df[['glasses']], y_pred_glasses , average="micro"))

print('F1 score (macro): ')
print(f1_score(df[['glasses']], y_pred_glasses , average="macro"))
print('F1 score (micro): ')
print(f1_score(df[['glasses']], y_pred_glasses , average="micro"))
print('F1 score (weighted): ')
print(f1_score(df[['glasses']], y_pred_glasses , average="weighted"))

print('Test accuracy: ')
print(test_acc)

1/1 - 2s - loss: 0.6711 - accuracy: 0.8000
These are the scores for the glasses attribute------------------------------------------------->
Precision score: 
0.8
Recall score: 
0.8
F1 score (macro): 
0.29629629629629634
F1 score (micro): 
0.8000000000000002
F1 score (weighted): 
0.7111111111111111
Test accuracy: 
0.800000011920929


4. Hair color Model:

In [ ]:
# here you need to specify the saved model from before
filepath = "/content/drive/MyDrive/VGG16-hair-color-10-0.3250.h5"

# load the model
model = load_model(filepath)


y_pred1_haircolor = model.predict(test_images)
y_pred_haircolor = np.argmax(y_pred1_haircolor, axis=1) 

test_loss, test_acc = model.evaluate(test_images,  df[['hair color']], verbose=2)

print('These are the scores for the Hair Color attribute------------------------------------------------->')
print('Precision score: ')
print(precision_score(df[['hair color']], y_pred_haircolor , average="micro"))
print('Recall score: ')
print(recall_score(df[['hair color']], y_pred_haircolor , average="micro"))

print('F1 score (macro): ')
print(f1_score(df[['hair color']], y_pred_haircolor , average="macro"))
print('F1 score (micro): ')
print(f1_score(df[['hair color']], y_pred_haircolor , average="micro"))
print('F1 score (weighted): ')
print(f1_score(df[['hair color']], y_pred_haircolor , average="weighted"))

print('Test accuracy: ')
print(test_acc)


1/1 - 1s - loss: 1.6690 - accuracy: 0.4000
These are the scores for the Hair Color attribute------------------------------------------------->
Precision score: 
0.4
Recall score: 
0.4
F1 score (macro): 
0.09523809523809525
F1 score (micro): 
0.4000000000000001
F1 score (weighted): 
0.2285714285714286
Test accuracy: 
0.4000000059604645


5. Hair top Model:

In [ ]:
# here you need to specify the saved model from before
filepath = "/content/drive/MyDrive/VGG16-hair-top-15-0.6450.h5"

# load the model
model = load_model(filepath)


y_pred1_hairtop = model.predict(test_images)
y_pred_hairtop = np.argmax(y_pred1_hairtop, axis=1) 

test_loss, test_acc = model.evaluate(test_images,  df[['hair top']], verbose=2)

print('These are the scores for the Hair Top attribute------------------------------------------------->')
print('Precision score: ')
print(precision_score(df[['hair top']], y_pred_hairtop , average="micro"))
print('Recall score: ')
print(recall_score(df[['hair top']], y_pred_hairtop , average="micro"))

print('F1 score (macro): ')
print(f1_score(df[['hair top']], y_pred_hairtop , average="macro"))
print('F1 score (micro): ')
print(f1_score(df[['hair top']], y_pred_hairtop , average="micro"))
print('F1 score (weighted): ')
print(f1_score(df[['hair top']], y_pred_hairtop , average="weighted"))


print('Test accuracy: ')
print(test_acc)

1/1 - 1s - loss: 0.7072 - accuracy: 0.7000
These are the scores for the Hair Top attribute------------------------------------------------->
Precision score: 
0.7
Recall score: 
0.7
F1 score (macro): 
0.6452991452991452
F1 score (micro): 
0.7
F1 score (weighted): 
0.667948717948718
Test accuracy: 
0.699999988079071


Exporting the predictions csv file as required and specified in the upload page for 'testing_code.ipynb':

In [ ]:
df_predictions = df[['file_name']]
df_predictions['wrinkles'] = y_pred_wrinkles
df_predictions['freakles'] = y_pred_freakles
df_predictions['glasses'] = y_pred_glasses
df_predictions['hair color'] = y_pred_haircolor
df_predictions['hair top'] = y_pred_hairtop

df_predictions.to_csv (r'/content/drive/MyDrive/predictions.csv', index = False, header=True)

df_predictions.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,file_name,wrinkles,freakles,glasses,hair color,hair top
0,image1.jpg,0,0,0,1,2
1,image2.jpg,0,0,0,1,3
2,image3.jpg,0,0,0,1,1
3,image4.jpg,0,0,0,1,2
4,image5.jpg,0,0,0,1,2
